In [2]:
# import library
import os
import random
# import cv2
import pandas as pd
import numpy as np

In [3]:
DATA_ROOT_TDCSFOG = './parkinson_data/train/tdcsfog/'
tdcsfog = pd.DataFrame()
for root, dirs, files in os.walk(DATA_ROOT_TDCSFOG):
    for name in files:       
        f = os.path.join(root, name)
        df_list= pd.read_csv(f)
        words = name.split('.')[0]
        df_list['file']= name.split('.')[0]
        tdcsfog = pd.concat([tdcsfog, df_list], axis=0)
tdcsfog

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,file
0,0,-9.494415,1.771898,-0.102501,0,0,0,8be08b9a99
1,1,-9.496606,1.767424,-0.088944,0,0,0,8be08b9a99
2,2,-9.497006,1.767036,-0.095913,0,0,0,8be08b9a99
3,3,-9.496839,1.769484,-0.098077,0,0,0,8be08b9a99
4,4,-9.494139,1.767601,-0.104659,0,0,0,8be08b9a99
5,5,-9.487846,1.771598,-0.109378,0,0,0,8be08b9a99
6,6,-9.496830,1.769499,-0.111628,0,0,0,8be08b9a99
7,7,-9.491668,1.767783,-0.124890,0,0,0,8be08b9a99
8,8,-9.487597,1.769563,-0.125089,0,0,0,8be08b9a99
9,9,-9.490366,1.775951,-0.125288,0,0,0,8be08b9a99


In [14]:
tdcsfog_metadata = pd.read_csv("./parkinson_data/tdcsfog_metadata.csv")
tdcsfog_metadata

,Id,Subject,Visit,Test,Medication
0,003f117e14,4dc2f8,3,2,on
1,009ee11563,f62eec,4,2,on
2,011322847a,231c3b,2,2,on
3,01d0fe7266,231c3b,2,1,off
4,024418ba39,fa8764,19,3,on
...,...,...,...,...,...
828,feba449e1a,312788,19,1,on
829,ff4f844fd3,af82b2,2,3,on
830,ff53514514,3b2403,2,3,on
831,ff92d9244d,2d57c2,20,2,on


In [15]:
tdcsfog_m= tdcsfog_metadata.merge(tdcsfog, how = 'inner', left_on = 'Id', right_on = 'file')
tdcsfog_m.drop(['file'], axis = 1, inplace = True)
tdcsfog_m

,Id,Subject,Visit,Test,Medication,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,003f117e14,4dc2f8,3,2,on,0,-9.533939,0.566322,-1.413525,0,0,0
1,003f117e14,4dc2f8,3,2,on,1,-9.536140,0.564137,-1.440621,0,0,0
2,003f117e14,4dc2f8,3,2,on,2,-9.529345,0.561765,-1.429332,0,0,0
3,003f117e14,4dc2f8,3,2,on,3,-9.531239,0.564227,-1.415490,0,0,0
4,003f117e14,4dc2f8,3,2,on,4,-9.540825,0.561854,-1.429471,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7062667,ffda8fadfd,7fcee9,20,1,off,4220,-9.403467,0.089003,-3.220304,0,0,0
7062668,ffda8fadfd,7fcee9,20,1,off,4221,-9.404246,0.090531,-3.216584,0,0,0
7062669,ffda8fadfd,7fcee9,20,1,off,4222,-9.405770,0.084380,-3.224039,0,0,0
7062670,ffda8fadfd,7fcee9,20,1,off,4223,-9.403579,0.084236,-3.236686,0,0,0


In [16]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [17]:
conditions = [
    (tdcsfog_m['StartHesitation'] == 1),
    (tdcsfog_m['Turn'] == 1),
    (tdcsfog_m['Walking'] == 1)]
choices = ['StartHesitation', 'Turn', 'Walking']
tdcsfog_m['event'] = np.select(conditions, choices, default='Normal')

In [18]:
tdcsfog_m['event'].value_counts().to_frame().style.background_gradient()

,event
Normal,4871262
Turn,1678782
StartHesitation,304790
Walking,207838


In [19]:
filtered_df = tdcsfog_m[tdcsfog_m['Id'] == "003f117e14"]
filtered_df

,Id,Subject,Visit,Test,Medication,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,event
0,003f117e14,4dc2f8,3,2,on,0,-9.533939,0.566322,-1.413525,0,0,0,Normal
1,003f117e14,4dc2f8,3,2,on,1,-9.536140,0.564137,-1.440621,0,0,0,Normal
2,003f117e14,4dc2f8,3,2,on,2,-9.529345,0.561765,-1.429332,0,0,0,Normal
3,003f117e14,4dc2f8,3,2,on,3,-9.531239,0.564227,-1.415490,0,0,0,Normal
4,003f117e14,4dc2f8,3,2,on,4,-9.540825,0.561854,-1.429471,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4677,003f117e14,4dc2f8,3,2,on,4677,-9.351431,0.370047,-2.286318,0,0,0,Normal
4678,003f117e14,4dc2f8,3,2,on,4678,-9.341410,0.347721,-2.321991,0,0,0,Normal
4679,003f117e14,4dc2f8,3,2,on,4679,-9.350667,0.361341,-2.297158,0,0,0,Normal
4680,003f117e14,4dc2f8,3,2,on,4680,-9.343977,0.365726,-2.292650,0,0,0,Normal


In [20]:
def analyze_one_exercise(df):
    conditions = [
        (df['StartHesitation'] == 1),
        (df['Turn'] == 1),
        (df['Walking'] == 1)]
    choices = ['StartHesitation', 'Turn', 'Walking']
    df['event'] = np.select(conditions, choices, default='Normal')
    return df['event'].value_counts()
    # return df['event'].value_counts().to_frame().style.background_gradient()

In [21]:
# Group the DataFrame by 'ID'
grouped = tdcsfog_m.groupby('Id')

In [37]:
threshold = 0.1
ids_to_remove = []

# Iterate over each group and create separate DataFrames for each ID
for id_, group in grouped:
    # Print the ID and the corresponding DataFrame
    # if id_ == "ffda8fadfd":
        
        # print(group)
        # print()
        aux_df = analyze_one_exercise(group)
        notMoves = aux_df[0]
        if len(aux_df) > 1:
            for index in range(1, len(aux_df)):
                moves = aux_df[index]
        else:
            moves = 0

        if (moves / notMoves) < threshold:
            ids_to_remove.append(id_)


In [38]:
# Remove the ids
filtered_df = tdcsfog_m[~tdcsfog_m['Id'].isin(ids_to_remove)]
filtered_df['event'].value_counts().to_frame().style.background_gradient()

,event
Normal,1185355
Turn,818732
StartHesitation,147228
Walking,106977


In [24]:
print("Normal df size = ", len(tdcsfog_m))
print("Filtered df size = ", len(filtered_df))

Normal df size =  7062672
Filtered df size =  525804


In [25]:
# Save the DataFrame to a CSV file
filtered_df.to_csv('filtered_data.csv', index=False)